In [2]:
import pandas as pd

In [3]:
OUT = pd.read_excel('/content/DADOS PARA A DIRF - trabalhando.xlsx', skiprows=1, index_col=0, sheet_name='OUT 2025')
AGO = pd.read_excel('/content/DADOS PARA A DIRF - trabalhando.xlsx', skiprows=1, index_col=0, sheet_name='AGO 2025')

In [4]:
OUT.head()

,CARTEIRA,DEPENDENCIA,NOME,CPF,NASCIMENTO,MENSALIDADE,CONSULTA,RETROATIVO RN,RETROATIVO,TOTAL
CÓD DA FAMILIA,,,,,,,,,,
90.0,1.371384e+15,Titular,ADAILTON SANTOS OLIVEIRA,738.717.296-04,06/03/1969,323.89,NaN,NaN,NaN,323.89
90.0,1.371384e+15,Esposa,APARECIDA RAIMUNDA DIAS OLIVEIRA,004.080.536-07,15/09/1970,323.89,NaN,NaN,NaN,323.89
90.0,1.371384e+15,Filho,ANDRE AYRES DIAS OLIVEIRA,131.684.916-32,30/09/2002,323.89,NaN,NaN,NaN,323.89
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251.0,1.371384e+15,Titular,ADAIR SALVADO JUNIOR,430.541.966-15,13/01/1962,743.94,NaN,NaN,NaN,743.94


In [6]:
def compare_months(df1, df2, month1_name, month2_name, gerar_df_entrada=False, gerar_df_saida=False):
    """
    Compara dois dataframes (meses) e identifica nomes presentes em um mas não no outro,
    ou presentes em ambos com diferença na coluna 'MENSALIDADE' (NaN vs não NaN),
    e imprime os resultados incluindo CPF, Nascimento e contagem por categoria.
    Opcionalmente, gera dataframes com os dados das pessoas que entraram e saíram.

    Args:
        df1 (pd.DataFrame): O primeiro dataframe (mês).
        df2 (pd.DataFrame): O segundo dataframe (mês).
        month1_name (str): O nome do primeiro mês para exibição.
        month2_name (str): O nome do segundo mês para exibição.
        gerar_df_entrada (bool): Se True, retorna um dataframe com as pessoas que entraram.
        gerar_df_saida (bool): Se True, retorna um dataframe com as pessoas que saíram.

    Returns:
        tuple: Uma tupla contendo os dataframes gerados, se solicitado.
               Retorna (df_entrada, None) se apenas gerar_df_entrada for True.
               Retorna (None, df_saida) se apenas gerar_df_saida for True.
               Retorna (df_entrada, df_saida) se ambos forem True.
               Retorna None se ambos forem False.
    """
    names_df1 = df1['NOME'].dropna().unique()
    names_df2 = df2['NOME'].dropna().unique()

    # Nomes em df1 que não estão em df2
    missing_in_df2 = []
    for name in names_df1:
        if name not in names_df2:
            person_data = df1[df1['NOME'] == name].iloc[0]
            missing_in_df2.append({
                'NOME': name,
                'CPF': person_data['CPF'],
                'NASCIMENTO': person_data['NASCIMENTO']
            })


    # Nomes em df2 que não estão em df1
    missing_in_df1 = []
    for name in names_df2:
        if name not in names_df1:
            person_data = df2[df2['NOME'] == name].iloc[0]
            missing_in_df1.append({
                'NOME': name,
                'CPF': person_data['CPF'],
                'NASCIMENTO': person_data['NASCIMENTO']
            })

    # Nomes presentes em ambos com diferença na coluna 'MENSALIDADE' (NaN vs não NaN)
    diff_mensalidade = []
    common_names = list(set(names_df1) & set(names_df2))

    for name in common_names:
        mensalidade_df1 = df1[df1['NOME'] == name]['MENSALIDADE'].iloc[0]
        mensalidade_df2 = df2[df2['NOME'] == name]['MENSALIDADE'].iloc[0]

        if pd.isna(mensalidade_df1) and not pd.isna(mensalidade_df2):
            person_data = df2[df2['NOME'] == name].iloc[0]
            diff_mensalidade.append({
                'NOME': f"{name}",
                'CPF': person_data['CPF'],
                'NASCIMENTO': person_data['NASCIMENTO']
            })
        elif not pd.isna(mensalidade_df1) and pd.isna(mensalidade_df2):
            person_data = df1[df1['NOME'] == name].iloc[0]
            diff_mensalidade.append({
                'NOME': f"{name}",
                 'CPF': person_data['CPF'],
                'NASCIMENTO': person_data['NASCIMENTO']
            })

    # Imprimindo os resultados
    print(f"Nomes em {month1_name} mas não em {month2_name} [{len(missing_in_df2)}]:")
    if missing_in_df2:
        for item in missing_in_df2:
            print(f"-Nome: {item['NOME']} - Cpf: {item['CPF']} - Nasc. {item['NASCIMENTO']}")
    else:
        print("- Nenhum")
    print("-" * 20)

    print(f"Pessoas que ENTRARAM [{len(missing_in_df1)}]:")
    if missing_in_df1:
        for item in missing_in_df1:
             print(f"-Nome: {item['NOME']} - Cpf: {item['CPF']} - Nasc. {item['NASCIMENTO']}")
    else:
        print("- Nenhum")
    print("-" * 20)

    print(f"Pessoas que SAÍRAM [{len(diff_mensalidade)}]:")
    if diff_mensalidade:
        for item in diff_mensalidade:
             print(f"-Nome: {item['NOME']} - Cpf: {item['CPF']} - Nasc. {item['NASCIMENTO']}")
    else:
        print("- Nenhum")
    print("-" * 20)
    print()

    # Gerar dataframes se solicitado
    df_entrada = pd.DataFrame(missing_in_df1) if gerar_df_entrada else None
    df_saida = pd.DataFrame(diff_mensalidade) if gerar_df_saida else None

    if gerar_df_entrada and gerar_df_saida:
        df_entrada.to_excel('entraram.xlsx', index=False)
        df_saida.to_excel('saida.xlsx', index=False)
    elif gerar_df_entrada:
        df_entrada.to_excel('entraram.xlsx', index=False)
    elif gerar_df_saida:
        df_saida.to_excel('saida.xlsx', index=False)
    else:
        return None

In [8]:
entrada = compare_months(AGO, OUT, "AGOSTO", "OUTUBRO", gerar_df_entrada=True)

Nomes em AGOSTO mas não em OUTUBRO [2]:
-Nome: PESSOAS QUE ENTRARAM ESSE MÊS - Nome - Cpf: CPF - Nasc. Nascimento
-Nome: PESSOAS QUE SAIRAM - Nome - Cpf: CPF - Nasc. Nascimento
--------------------
Pessoas que ENTRARAM [10]:
-Nome: ANA PAULA MONI SILVA - Cpf: 038.521.926-17 - Nasc. 24/07/1977
-Nome: ANDRE LUIZ DOS SANTOS - Cpf: 025.785.606-41 - Nasc. 12/01/1977
-Nome: AUGUSTO MONI SANTOS - Cpf: 140.799.916-81 - Nasc. 27/08/2013
-Nome: DIEGO HENRIQUE SAMPAIO DE SOUZA - Cpf: 016.386.526-43 - Nasc. 12/01/1988
-Nome: ISABELA GUIMARAES DIAS - Cpf: 056.393.526-04 - Nasc. 15/09/2025
-Nome: JOAO DA MATTA ARAUJO - Cpf: 080.799.356-53 - Nasc. 08/02/1937
-Nome: ELZA GUIMARAES DE ARAUJO - Cpf: 607.316.566-87 - Nasc. 11/09/1939
-Nome: RAFAEL MARIO DA SILVA - Cpf: 114.150.066-36 - Nasc. 10/11/1992
-Nome: TARCISO JOAQUIM DA SILVA - Cpf: 352.235.026-04 - Nasc. 25/11/1957
-Nome: MARIA DULCIENE DE JESUS DA SILVA - Cpf: 495.510.946-20 - Nasc. 03/12/1960
--------------------
Pessoas que SAÍRAM [3]:
-Nome: